# libCEED for Python examples

This is a tutorial to illustrate the main feautures of the Python interface for [libCEED](https://github.com/CEED/libCEED/), the low-level API library for efficient high-order discretization methods developed by the co-design [Center for Efficient Exascale Discretizations](https://ceed.exascaleproject.org/) (CEED) of the [Exascale Computing Project](https://www.exascaleproject.org/) (ECP).

While libCEED's focus is on high-order finite/spectral element method implementations, the approach is mostly algebraic and thus applicable to other discretizations in factored form, as explained in the [user manual](https://libceed.readthedocs.io/) and [Doxygen documentation](https://codedocs.xyz/CEED/libCEED/).

## Setting up libCEED for Python

Clone or download libCEED by running

In [ ]:
! git clone https://github.com/CEED/libCEED.git

Then move to the libCEED folder

In [ ]:
cd libCEED

And compile the base library by running

In [ ]:
! make

Build libCEED for Python by running

In [ ]:
python setup.py build install

Or without superuser permissions

In [ ]:
python setup.py build install --user

## CeedQFunction

Here we show some basic examples to illustrate the `CeedQFunction` class. In libCEED, QFunctions represent the spatial terms of the point-wise functions describing the physics at the quadrature points (see [the API documentation](https://libceed.readthedocs.io/en/latest/libCEEDapi.html#api-description)).

* In the following example, we create and evaluate a QFunction (for the mass operator in 1D) from the gallery of available built-in QFunctions in libCEED.

In [13]:
import libceed
import numpy as np

ceed = libceed.Ceed()

qf_setup = ceed.QFunctionByName("Mass1DBuild")
qf_mass = ceed.QFunctionByName("MassApply")

q = 8

j_array = np.zeros(q, dtype="float64")
w_array = np.zeros(q, dtype="float64")
u_array = np.zeros(q, dtype="float64")
v_true  = np.zeros(q, dtype="float64")
for i in range(q):
  x = 2.*i/(q-1) - 1
  j_array[i] = 1
  w_array[i] = 1 - x*x
  u_array[i] = 2 + 3*x + 5*x*x
  v_true[i]  = w_array[i] * u_array[i]

j = ceed.Vector(q)
j.set_array(j_array, cmode=libceed.USE_POINTER)
w = ceed.Vector(q)
w.set_array(w_array, cmode=libceed.USE_POINTER)
u = ceed.Vector(q)
u.set_array(u_array, cmode=libceed.USE_POINTER)
v = ceed.Vector(q)
v.set_value(0)
qdata = ceed.Vector(q)
qdata.set_value(0)

inputs = [ j, w ]
outputs = [ qdata ]
qf_setup.apply(q, inputs, outputs)

inputs = [ w, u ]
outputs = [ v ]
qf_mass.apply(q, inputs, outputs)

v_array = v.get_array_read()

print(v_array)

v.restore_array_read()

[0.         1.17950854 1.33277801 1.63931695 2.4789671  3.43190337
 3.2786339  0.        ]


* In the following example, we create and evaluate a built-in identity QFunction.

In [14]:
qf = ceed.IdentityQFunction(1, libceed.EVAL_INTERP, libceed.EVAL_INTERP)

q = 8

u_array = np.zeros(q, dtype="float64")
for i in range(q):
  u_array[i] = i*i

u = ceed.Vector(q)
u.set_array(u_array, cmode=libceed.USE_POINTER)
v = ceed.Vector(q)
v.set_value(0)

inputs = [ u ]
outputs = [ v ]
qf.apply(q, inputs, outputs)

v_array = v.get_array_read()

print(v_array)

v.restore_array_read()

[ 0.  1.  4.  9. 16. 25. 36. 49.]


* In the following example, we create and evaluate a built-in identity QFunction with size > 1.

In [16]:
size = 3

qf = ceed.IdentityQFunction(size, libceed.EVAL_INTERP, libceed.EVAL_INTERP)

q = 8

u_array = np.zeros(q*size, dtype="float64")
for i in range(q*size):
  u_array[i] = i*i

u = ceed.Vector(q*size)
u.set_array(u_array, cmode=libceed.USE_POINTER)
v = ceed.Vector(q*size)
v.set_value(0)

inputs = [ u ]
outputs = [ v ]
qf.apply(q, inputs, outputs)

v_array = v.get_array_read()

print(v_array)

v.restore_array_read()

[  0.   1.   4.   9.  16.  25.  36.  49.  64.  81. 100. 121. 144. 169.
 196. 225. 256. 289. 324. 361. 400. 441. 484. 529.]


* In the following example, we create and view two QFunctions (for the setup and apply, respectively, of the mass operator in 1D) from the gallery of available built-in QFunctions in libCEED

In [17]:
qf_setup = ceed.QFunctionByName("Mass1DBuild")
qf_mass = ceed.QFunctionByName("MassApply")

print(qf_setup)
print(qf_mass)


Gallery CeedQFunction Mass1DBuild
  2 Input Fields:
    Input Field [0]:
      Name: "dx"
      Size: 1
      EvalMode: "gradient"
    Input Field [1]:
      Name: "weights"
      Size: 1
      EvalMode: "quadrature weights"
  1 Output Field:
    Output Field [0]:
      Name: "qdata"
      Size: 1
      EvalMode: "none"

Gallery CeedQFunction MassApply
  2 Input Fields:
    Input Field [0]:
      Name: "u"
      Size: 1
      EvalMode: "interpolation"
    Input Field [1]:
      Name: "qdata"
      Size: 1
      EvalMode: "none"
  1 Output Field:
    Output Field [0]:
      Name: "v"
      Size: 1
      EvalMode: "interpolation"

